<a href="https://colab.research.google.com/github/NaziaToma/BugType-and-BugFix-Predictor/blob/main/BugType_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nltk scikit-learn

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
# Downloading necessary NLTK datasets
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Initializing nltk's lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lemmatize each word
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

In [ ]:
# Loading dataset
data = pd.read_csv('/content/drive/MyDrive/bug_type.csv')

In [ ]:
import re
# Function to clean text data
def clean_text(text):
    # Removing special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    return text

In [ ]:
#Applying clean text function
data['Short Description'] = data['Short Description'].apply(clean_text)
data['Bug Report'] = data['Bug Report'].apply(clean_text)

In [ ]:
# Combining Short Description and Bug Report into a single text feature
data['text'] = data['Short Description'] + ' ' + data['Bug Report']

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Encoding the 'Bug Type' labels into numerical values
label_encoder = LabelEncoder()
data['Type_encoded'] = label_encoder.fit_transform(data['Bug Type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initializing Label Encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y = label_encoder.fit_transform(data['Bug Type'])


In [ ]:
# Print the original class names for labels 0, 1, and 2
print(label_encoder.inverse_transform([0, 1, 2]))

['Concurrency' 'Other' 'Performance']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(data['text'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Initializing the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Training the model
nb_classifier.fit(X_train, y_train)


MultinomialNB()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Prediction on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred) * 100
print(f'Accuracy: {accuracy:.2f}%')

report = classification_report(y_test, y_pred, output_dict=True)
print("Classification Report:")
for label, metrics in report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f'Class {label} - Precision: {metrics["precision"] * 100:.2f}%, Recall: {metrics["recall"] * 100:.2f}%, F1-Score: {metrics["f1-score"] * 100:.2f}%')
    elif label == 'accuracy':
        continue
    else:
        print(f'{label.capitalize()} - Precision: {metrics["precision"] * 100:.2f}%, Recall: {metrics["recall"] * 100:.2f}%, F1-Score: {metrics["f1-score"] * 100:.2f}%')


Accuracy: 69.37%
Classification Report:
Class 0 - Precision: 100.00%, Recall: 2.70%, F1-Score: 5.26%
Class 1 - Precision: 78.75%, Recall: 72.41%, F1-Score: 75.45%
Class 2 - Precision: 63.83%, Recall: 91.84%, F1-Score: 75.31%
Macro avg - Precision: 80.86%, Recall: 55.65%, F1-Score: 52.01%
Weighted avg - Precision: 75.71%, Recall: 69.37%, F1-Score: 63.69%
